In [ ]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from pygam import LinearGAM, s
import seaborn as sns


warnings.filterwarnings("ignore")




## Load data

In [ ]:
data = pd.read_csv('data/BBAG-cross.csv')
data = data[data.country != 'Slovakia'].reset_index(drop =  True)

In [ ]:
data

## Dispersion

In [ ]:


vars_ = ["Mono"]

df = data.copy()
df = df[["country"] + vars_].dropna(subset=["country"]).copy()

top_n = 10          
disp_metric = "iqr" 

df_long = df.melt(id_vars="country", value_vars=vars_, var_name="Feature", value_name="Value")
df_long = df_long.dropna(subset=["Value"]).copy()

def get_order(d, metric="iqr", top_n=20):
    g = d.groupby("country")["Value"]
    if metric == "std":
        disp = g.std()
    else:
        disp = g.quantile(0.75) - g.quantile(0.25)
    order = disp.sort_values(ascending=False).head(top_n).index.tolist()
    return order

sns.set_theme(style="whitegrid", context="talk")

nrows, ncols = 1, 2
fig, axes = plt.subplots(nrows, ncols, figsize=(15, 5), sharey=False)

axes = axes.flatten()

for ax, feat in zip(axes, vars_):
    d = df_long[df_long["Feature"] == feat].copy()
    order = get_order(d, metric=disp_metric, top_n=top_n)
    d = d[d["country"].isin(order)]
    d["country"] = pd.Categorical(d["country"], categories=order, ordered=True)

    sns.boxplot(
        data=d, x="country", y="Value",
        ax=ax, width=0.6
    )
    sns.stripplot(
        data=d, x="country", y="Value",
        ax=ax, size=2.5, alpha=0.35, jitter=0.25
    )

    #ax.set_title(f"{feat} (Top {top_n} dispersion)")
    ax.set_xlabel("")
    ax.set_ylabel(feat)
    ax.tick_params(axis="x", labelrotation=90)
    ax.grid(False)
    

for ax in axes[len(vars_):]:
    ax.axis("off")

plt.tight_layout()


plt.show()
